## Day 18

### Part 1

In [204]:
from collections import defaultdict

def regOrValue(c,reg):
    try:
        v = int(c)
    except:
        v = reg[c]
    return v

def duet(prog,verbose=False): 
    reg = defaultdict(lambda: 0)
    i = 0
    sound = 0
    while True:
        if i>=len(prog):
            break
        cmd = prog[i].split(" ")
        if verbose:
            print(cmd)
        if cmd[0]=="snd":
            sound = reg[cmd[1]]
        elif cmd[0]=="set":
            reg[cmd[1]]  = regOrValue(cmd[2],reg)
        elif cmd[0]=="add":
            reg[cmd[1]] += regOrValue(cmd[2],reg)
        elif cmd[0]=="mul":
            reg[cmd[1]] *= regOrValue(cmd[2],reg)
        elif cmd[0]=="mod":
            reg[cmd[1]] %= regOrValue(cmd[2],reg)
        elif cmd[0]=="rcv":
            if reg[cmd[1]]:
                if verbose:
                    print("Recovering",sound)
                if sound > 0:
                    return sound
        elif cmd[0]=="jgz":
            if regOrValue(reg[cmd[1]],reg)>0:
                i += regOrValue(cmd[2],reg) - 1 # will increment by 1 as for all other actions
        i += 1
    return reg

In [205]:
prog = [
"set a 1",
"add a 2",
"mul a a",
"mod a 5",
"snd a",
"set a 0",
"rcv a",
"jgz a -1",
"set a 1",
"jgz a -2"
]

In [206]:
duet(prog,verbose=True) 

['set', 'a', '1']
['add', 'a', '2']
['mul', 'a', 'a']
['mod', 'a', '5']
['snd', 'a']
['set', 'a', '0']
['rcv', 'a']
['jgz', 'a', '-1']
['set', 'a', '1']
['jgz', 'a', '-2']
['jgz', 'a', '-1']
['rcv', 'a']
Recovering 4


4

In [200]:
with open("data/input18.txt") as f:
    prog = [ l.strip("\n") for l in f.readlines() ]
    
print("Part 1:",duet(prog)) 

Part 1: 8600


### Part 2

In [207]:
from collections import defaultdict

def regOrValue(c,reg):
    try:
        v = int(c)
    except:
        v = reg[c]
    return v

class Program:
    
    def __init__(self,ID,prog):
    
        self.ID   = ID
        self.prog = list(prog)
        self.reg  = defaultdict(int)
        self.reg['p'] = ID
        self.i = 0
        self.sounds = 0
        self.received = []
        self.isWaiting = False
        self.reachedEnd = False

    def queueSound(self,s):
        self.received.append(s)
        self.isWaiting = False
        
    def Run(self,insound=0): 
        
        self.isWaiting = False
        
        while True:
            
            if self.i>=len(prog):
                print("Reached end of program",self.ID)
                self.reachedEnd = True
            
            cmd = self.prog[self.i].split(" ")
            
            if cmd[0]=="snd":
                self.i += 1 # increment here since returning before global increment
                self.sounds += 1
                return regOrValue(cmd[1],self.reg)
            elif cmd[0]=="set":
                self.reg[cmd[1]]  = regOrValue(cmd[2],self.reg)
            elif cmd[0]=="add":
                self.reg[cmd[1]] += regOrValue(cmd[2],self.reg)
            elif cmd[0]=="mul":
                self.reg[cmd[1]] *= regOrValue(cmd[2],self.reg)
            elif cmd[0]=="mod":
                self.reg[cmd[1]] %= regOrValue(cmd[2],self.reg)
            elif cmd[0]=="rcv":
                if len(self.received)>0:
                    self.reg[cmd[1]] = self.received.pop(0)
                else:
                    # no local increment of i, since I want this instruction to be re-executed is input gets available
                    self.isWaiting = True
                    return None
            elif cmd[0]=="jgz":
                if regOrValue(cmd[1],self.reg)>0:
                    self.i += regOrValue(cmd[2],self.reg) - 1
            
            self.i += 1


def Duet(prog):
    
    p1 = Program(0,prog)
    p2 = Program(1,prog)
    
    while True:
    
        # Run P1 until it cannot run anymore because is waiting to receive sounds
        while not p1.isWaiting:
            o1 = p1.Run()
            if o1 != None:
                p2.queueSound(o1) # store the emitted sounds in P2 queue
        
        # Run P2 until it cannot run anymore because is waiting to receive sounds
        while not p2.isWaiting:
            o2 = p2.Run()
            if o2 != None:
                p1.queueSound(o2) # store the emitted sounds in P1 queue
    
        # Both programs are stuck, end the execution
        if p1.isWaiting and p2.isWaiting:
            break

        # At least one program has reached the end of the instructions, end the execution
        if p1.reachedEnd or p2.reachedEnd:
            break
            
    return p1,p2

In [208]:
prog = [
"snd 1",
"snd 2",
"snd p",
"rcv a",
"rcv b",
"rcv c",
"rcv d"
]

p1,p2 = Duet(prog)
print(p2.sounds)

3


In [209]:
with open("data/input18.txt") as f:
    prog = [ l.strip("\n") for l in f.readlines() ]

p1,p2 = Duet(prog)
print("Part 2:",p2.sounds)

Part 2: 7239
